<br><br>
<font color='FireBrick' size='7'><b> Double/Debiased Machine Learning (DML)</b></font>

<font color='black' size='5'><b>using IPUMS data</b></font>

In [1]:
import pandas as pd

main_data_df = pd.read_feather("/Users/dadmehr/R/main_data.feather")

print(f"Number of rows: {main_data_df.shape[0]}")
print(f"Number of columns: {main_data_df.shape[1]}")

Number of rows: 43424806
Number of columns: 54


In [2]:
#data = data.sample(frac=0.01)  # Sampling 1% of the dataset due to GitHub's 25MB file size limit for uploads.

In [3]:
data = main_data_df[main_data_df['YEAR'] == 2021]
data = data[['INCTOT', 'SEX', 'EDUC', 'EDUCD', 'RACE', 'AGE','EMPSTAT', 'MARST']]

data.to_csv('IPUMS_2021_data.csv', index=False)

In [4]:
data

,INCTOT,SEX,EDUC,EDUCD,RACE,AGE,EMPSTAT,MARST
40172207,15000.0,1,7,71,1,85,3,2
40172208,4800.0,2,6,63,2,67,3,5
40172209,36000.0,1,7,71,1,74,3,5
40172210,0.0,1,3,30,1,16,3,6
40172211,7200.0,1,6,63,1,83,3,4
...,...,...,...,...,...,...,...,...
43424801,72300.0,1,6,65,1,77,3,1
43424802,9700.0,2,6,64,1,72,3,1
43424803,16300.0,1,6,64,1,49,1,4
43424804,97000.0,1,8,81,1,64,3,1


In [5]:
#!pip install statsmodels econml

In [6]:
import statsmodels.api as sm

# Prepare the variables
X = data[['SEX', 'AGE']]  # Independent variables
y = data['INCTOT']  # Dependent variable

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary of the regression model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                 INCTOT   R-squared:                       0.404
Model:                            OLS   Adj. R-squared:                  0.404
Method:                 Least Squares   F-statistic:                 1.100e+06
Date:                Tue, 23 Apr 2024   Prob (F-statistic):               0.00
Time:                        11:29:56   Log-Likelihood:            -5.2898e+07
No. Observations:             3252599   AIC:                         1.058e+08
Df Residuals:                 3252596   BIC:                         1.058e+08
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.613e+06   5552.345   1010.906      0.0

In [7]:
import doubleml as dml
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load your data (replace 'your_data.csv' with your actual file name)

# Get the column names for the outcome, treatment, and control(s)
y_col = 'INCTOT'  # Outcome variable
d_col = 'SEX'  # Treatment variable
x_cols = ['AGE']  # Control variables

# Create the dataset for DoubleML
data_dml = dml.DoubleMLData(data, y_col=y_col, d_cols=[d_col], x_cols=x_cols)

# Initialize the machine learning algorithms for DML
ml_y = RandomForestRegressor()  # For the outcome variable
ml_d = RandomForestRegressor()  # For the treatment variable

# Initialize and fit the DoubleMLPLR model
dml_plr = dml.DoubleMLPLR(data_dml, ml_y, ml_d, n_folds=3)
dml_plr.fit()

# Retrieve the summary DataFrame
summary = dml_plr.summary

# Check the available column names
print(summary.columns)

# Extract the coefficient for the treatment variable (SEX)
sex_coef = summary['coef'].loc[d_col]

# Output the coefficient for SEX
print(f"The estimated coefficient for SEX is: {sex_coef}")

Index(['coef', 'std err', 't', 'P>|t|', '2.5 %', '97.5 %'], dtype='object')
The estimated coefficient for SEX is: -17974.027776317907
